# 数据加载
## 文件地址

In [21]:
import os
import collections
ROOT = os.getcwd()
ROOT = os.path.split(ROOT)[0]
ROOT = os.path.join(ROOT,'data_set')

class Ml_100K():
    #下载地址：https://github.com/rexrex9/kb4recMovielensDataProcess
    # 以双下划线（__）开头的属性名（如这里的__BASE）在类定义里通常会被解释器进行名称改写（Name Mangling）
    # 用于实现一种有限的 “私有” 机制，实际的属性名会被改写成_类名__属性名的形式。
    __BASE = os.path.join(ROOT, 'ml-100k')
    ORGINAL_DIR = os.path.join(ROOT,'ml-100k-orginal')
    USER_DF = os.path.join(ORGINAL_DIR,'user_df.csv')
    ITEM_DF = os.path.join(ORGINAL_DIR,'item_df.csv')
    ITEM_DF_0 = os.path.join(ORGINAL_DIR, 'item_df_0.csv')

    KG=os.path.join(__BASE,'kg_index.tsv')
    RATING = os.path.join(__BASE,'rating_index.tsv')
    RATING5 = os.path.join(__BASE, 'rating_index_5.tsv')

class Ml_latest_small():
    __BASE = os.path.join(ROOT,'ml-latest-small')
    RATING_TS = os.path.join(__BASE,'rating_index_ts.tsv')
    SEQS = os.path.join(__BASE, 'seqs.npy')

    SEQS_NEG = os.path.join(__BASE, 'seqsWithNeg.npy')
print(ROOT)
print(Ml_100K._Ml_100K__BASE)
print(Ml_100K.RATING)

d:\DeepLearning\推荐系统\recbyhand\data_set
d:\DeepLearning\推荐系统\recbyhand\data_set\ml-100k
d:\DeepLearning\推荐系统\recbyhand\data_set\ml-100k\rating_index.tsv


In [22]:
import json

def readTriple(path,sep=None):
    # with语句的作用：自动管理文件资源
    # path是路径 ‘r’是read表示只读模式  utf-8是文件的编码格式
    # 文件对象被赋值给f
    with open(path,'r',encoding='utf-8') as f:
        # readlines()方法会一次性读取文件中所有的行
        # 并将每行内容作为一个字符串元素储存在一个列表中
        for line in f.readlines():
            if sep:
                # .strip()去除字符串两端的空白字符
                # .split()按照分隔符将字符串拆分成列表
                lines = line.strip().split(sep)
            else:
                lines=line.strip().split()
            # 如果长度不是3那就跳过本次循环
            if len(lines)!=3:continue
            # 长度是3 利用yield将Lines列表作为一个生产的数据项返回
            yield lines

In [23]:
import random
from tqdm import tqdm

def readRecData(path,test_ratio = 0.1):
    print('读取用户评分三元组...')
    user_set,item_set=set(),set()
    triples=[]
    for u, i, r in tqdm(readTriple(path)):
        user_set.add(int(u))
        item_set.add(int(i))
        triples.append((int(u),int(i),int(r)))

    test_set=random.sample(triples,int(len(triples)*test_ratio))
    train_set=list(set(triples)-set(test_set))

    #返回用户集合列表，物品集合列表，与用户，物品，评分三元组列表
    return list(user_set),list(item_set),train_set,test_set


# 近邻协同过滤
## UerCF_01

In [24]:
# 得到用户-物品集合
def getSet( triples ):
    user_items = collections.defaultdict( set )
    for u, i, r in triples:
        if r == 1:
            user_items[u].add(i)
    return user_items

# 两个集合间的cos相似度
def cos4set( set1, set2 ):
    return len(set1&set2)/(len(set1)*len(set2))**0.5

#knn算法
def knn4set(trainset, k, sim_method):
    '''
    :param trainset: 训练集合
    :param k: 近邻数量
    :param sim_method: 相似度方法
    :return: {样本1:[近邻1,近邻2，近邻3]}
    '''
    sims = {}
    #两个for循环遍历训练集合
    for e1 in tqdm(trainset):
        ulist = []#初始化一个列表来记录样本e1的近邻
        for e2 in trainset:
            #如果两个样本相同则跳过
            if e1 == e2 or \
                    len(trainset[e1]&trainset[e2]) == 0:
                #如果两个样本的交集为0也跳过
                continue
            #用相似度方法取得两个样本的相似度
            sim = sim_method(trainset[e1], trainset[e2])
            ulist.append((e2, sim))
        #排序后取前K的样本
        sims[e1] = [i[0] for i in sorted(ulist, key=lambda x:x[1], reverse=True)[:k]]
    return sims

#得到基于相似用户的推荐列表
def get_recomedations_by_usrCF( user_sims, user_o_set ):
    '''
    :param user_sims: 用户的近邻集:{样本1:[近邻1,近邻2，近邻3]}
    :param user_o_set: 用户的原本喜欢的物品集合:{用户1:{物品1,物品2，物品3}}
    :return: 每个用户的推荐列表{用户1:[物品1，物品2，物品3]}
    '''
    recomedations = collections.defaultdict(set)
    for u in user_sims:
        for sim_u in user_sims[u]:
            #将近邻用户喜爱的电影与自己观看过的电影去重后推荐给自己
            recomedations[u] |= (user_o_set[sim_u] - user_o_set[u])
    return recomedations

#得到基于UserCF的推荐列表
def trainUserCF( user_items, sim_method, k = 5 ):
    user_sims = knn4set( user_items, k, sim_method )
    recomedations = get_recomedations_by_usrCF( user_sims, user_items )
    return recomedations

In [25]:
_, _, train_set, test_set = readRecData( Ml_100K.RATING, test_ratio=0.1 )

读取用户评分三元组...


97466it [00:00, 1324299.66it/s]


In [26]:
user_items = getSet( train_set )
recomedations_by_userCF = trainUserCF( user_items, cos4set, k=5 )

  0%|          | 0/942 [00:00<?, ?it/s]

100%|██████████| 942/942 [00:01<00:00, 751.63it/s]


In [27]:
print(recomedations_by_userCF)

defaultdict(<class 'set'>, {363: {640, 1155, 389, 133, 973, 846, 845, 717, 404, 1431, 151, 1116, 1123, 1444, 805, 806, 1577, 813, 500, 1273}, 232: {1, 4, 7, 15, 16, 17, 18, 1047, 23, 1049, 27, 1052, 32, 33, 1058, 1061, 45, 1070, 1073, 1079, 57, 1081, 1088, 68, 1094, 1103, 1104, 1106, 1110, 87, 94, 1127, 1128, 105, 1130, 1132, 110, 1135, 1136, 1137, 116, 117, 1149, 1155, 1160, 138, 1171, 1180, 159, 1184, 165, 166, 1192, 168, 1193, 171, 169, 178, 1203, 1202, 179, 182, 185, 1209, 1213, 192, 1221, 201, 1226, 1229, 1238, 216, 1247, 226, 1254, 233, 236, 1273, 250, 255, 256, 1282, 1283, 263, 1294, 1296, 272, 273, 274, 1300, 278, 1303, 282, 1306, 1308, 284, 285, 1313, 294, 300, 304, 1340, 318, 1344, 333, 1367, 344, 345, 1379, 1382, 359, 361, 1394, 371, 1395, 372, 370, 375, 1400, 382, 1407, 1410, 387, 389, 1414, 1421, 399, 400, 1425, 402, 1431, 408, 1448, 1451, 1452, 430, 436, 1468, 445, 1469, 453, 456, 1481, 1482, 1485, 462, 463, 1490, 1494, 471, 472, 1499, 476, 480, 1510, 1516, 497, 1521, 499

In [28]:
def getSet( triples ):
    #已物品为索引，喜欢物品的用户集
    item_users = collections.defaultdict( set )
    user_items = collections.defaultdict( set )
    for u, i, r in triples:
        if r == 1:
            item_users[i].add(u)
            user_items[u].add(i)
    return item_users,user_items

#得到基于相似物品的推荐列表
def get_recomedations_by_itemCF( item_sims, user_o_set ):
    '''
    :param item_sims: 物品的近邻集:{样本1:[近邻1,近邻2，近邻3]}
    :param user_o_set: 用户的原本喜欢的物品集合:{用户1:{物品1,物品2，物品3}}
    :return: 每个用户的推荐列表{用户1:[物品1，物品2，物品3]}
    '''
    recomedations = collections.defaultdict(set)
    for u in user_o_set:
        for item in user_o_set[u]:
            # 将自己喜欢物品的近邻物品与自己观看过的视频去重后推荐给自己
            if item in item_sims:
                recomedations[u] |= set( item_sims[item] ) - user_o_set[u]
    return recomedations

#得到基于ItemCF的推荐列表
def trainItemCF( item_users, sim_method, user_items, k = 5 ):
    item_sims = knn4set( item_users, k, sim_method )
    recomedations = get_recomedations_by_itemCF( item_sims, user_items )
    return recomedations

In [29]:
_, _, train_set, test_set = readRecData( Ml_100K.RATING, test_ratio=0.1 )

读取用户评分三元组...


97466it [00:00, 1255438.11it/s]


In [30]:
item_users,user_items = getSet( train_set )
recomedations_by_itemCF = trainItemCF( item_users, cos4set, user_items, k=5 )

100%|██████████| 1359/1359 [00:01<00:00, 1026.88it/s]
